#### Density matrix
A quantum state can be represented as a vector in Hilbert Space $\mathcal{H}$  
However in the most general scenario, they are represented by a density operator $\rho$ acting on $\mathcal{H}$  
$$\rho = \sum_{i=1}^N p_{i}\ket{i}\bra{i}$$

Here $\ket{i}$ is the state prepared with probability $p_{i}$
The states are normalized but not necessarily independent.  
The matrix form of $\rho$ is known as the _**density matrix**_


In [26]:
import pennylane as qml
import numpy as np
from IPython.display import Latex

#### Solution to Codercise N.1.1

In [27]:
def build_density_matrix(state_1: np.array(complex), state_2: np.array(complex), p_1: float, p_2: float):
    projector_1 = np.outer(state_1, np.conjugate(state_1))
    projector_2 =np.outer(state_2, np.conjugate(state_2))
    
    probability_1 = p_1 * projector_1
    probability_2 = p_2 * projector_2
    
    density_matrix = probability_1 + probability_2
    
    return density_matrix

In [28]:
print("state_1 = |+y>, state_2 = |+x>, p_1 = 0.5, p_2 = 0.5")
print("density_matrix:")
print(build_density_matrix([1,1j]/np.sqrt(2), [1,1]/np.sqrt(2), 0.5, 0.5))

state_1 = |+y>, state_2 = |+x>, p_1 = 0.5, p_2 = 0.5
density_matrix:
[[0.5 +0.j   0.25-0.25j]
 [0.25+0.25j 0.5 +0.j  ]]


### Eqn (1) is defined as  
$$\rho = \sum_{i=1}^N p_{i}\ket{i}\bra{i}$$  
Not every matrix has this form. In order for a matrix to represent this form it must satisfy the following properties:
1. $\rho$ is Hermitian, i.e. $\rho^\dagger = \rho$
2. $\rho$ has unit trace, i.e. $\mathrm{tr}{\rho} = 1$
3. $\rho$ is positive semidefinite, i.e. all of its eigenvalues are positive 

##### Solution to Codercise N.1.2.a

In [43]:
def is_hermitian(matrix: np.array(np.array(complex))) -> bool:
    """
    Checks if the input matrix is hermitian or not
    """
    return np.array_equal(matrix, matrix.conj().T)

In [44]:
"""
Test the is_hermitian method
"""
matrix_1 = np.array([[1,1j],[-1j,1]])
matrix_2 = np.array([[1,2],[3,4]])

print("Is matrix [[1,1j],[-1j,1]] Hermitian?")
print(is_hermitian(matrix_1))
print("Is matrix [[1,2],[3,4]] Hermitian?")
print(is_hermitian(matrix_2))

Is matrix [[1,1j],[-1j,1]] Hermitian?
True
Is matrix [[1,2],[3,4]] Hermitian?
False


##### Solution to Codercise N.1.2.b

In [45]:
def has_trace_one(matrix: np.array(np.array(complex))) -> bool:
    """
    Checks if the input matrix has unit trace
    """
    return np.trace(matrix) == 1

In [46]:
"""
Test the has_trace_one method
"""
matrix_1 = [[1/2,1j],[-1j,1/2]]
matrix_2 = [[1,2],[3,4]]
    
print("Does [[1/2,1j],[-1j,1/2]] have unit trace?")
print(has_trace_one(matrix_1))
print("Does [[1,2],[3,4]] have unit trace?")
print(has_trace_one(matrix_2))

Does [[1/2,1j],[-1j,1/2]] have unit trace?
True
Does [[1,2],[3,4]] have unit trace?
False


##### Solution to Codercise 4.1.2.c

In [47]:
def is_semi_positive(matrix: np.array(np.array(complex))) -> bool:
    """
    Checks if the input matrix is semidefinite positive, i.e. all of its eignevales are positive.
    [NOTE]: I think this method be renamed to is_semidefinite_positive
    """
    return all(np.linalg.eigvals(matrix) >= 0)

In [48]:
"""
Test the is_semi_positive method
"""
matrix_1 = [[3/4,1/4],[1/4,1/4]]
matrix_2 = [[0,1/4],[1/4,1/4]]

print("Is matrix [[3/4,1/4],[1/4,1/4]] positive semidefinite?")
print(is_semi_positive(matrix_1))
print("Is matrix [[0,1/4],[1/4,1/4]] positive semidefinite?")
print(is_semi_positive(matrix_2))

Is matrix [[3/4,1/4],[1/4,1/4]] positive semidefinite?
True
Is matrix [[0,1/4],[1/4,1/4]] positive semidefinite?
False


In [49]:
def is_density_matrix(matrix: np.array(np.array(complex))) -> bool:
    """
    Checks if the input matrix satisfies all 3 conditions to be a denisty matrix
    """
    return is_hermitian(matrix) and has_trace_one(matrix) and is_semi_positive(matrix)

In [50]:
"""
Test for the is_density_matrix method
"""
matrix_1 = np.array([[3/4,0.25j],[-0.25j,1/4]])
matrix_2 = np.array([[0,1/4],[1/4,1/4]])
    
print("Is [[3/4,0.25j],[-0.25j,1/4]] a density matrix?")
print(is_density_matrix(matrix_1))
print("Is matrix [[0,1/4],[1/4,1/4]] a density matrix?")
print(is_density_matrix(matrix_2))

Is [[3/4,0.25j],[-0.25j,1/4]] a density matrix?
True
Is matrix [[0,1/4],[1/4,1/4]] a density matrix?
False


#### Mixed States

In the event that our state preparation results in the state $\ket{\psi}$ with _*certainty*_, i.e. with probability $1$  
Then we say that this state is a _**pure state**_ and the density operator is given as $$\rho = \ket{\psi}\bra{\psi}$$  

If the state prepared is not a pure state, then we call it a _**mixed state**_  

One way to determine whether or not a density matrix represents a pure state or a mixed state, is to use the purity of the matrix  
$$\gamma{(\rho)} = \mathrm{tr}{\rho^2}$$

The state is pure if and only if $\gamma{(\rho)} = 1$

In [51]:
def purity(density_matrix: np.array(np.array(complex))) -> float:
    """
    Calculate the square of the trace of the matrix
    """
    trace = np.linalg.norm(density_matrix) #Calculate the trace of the matrix
    return np.square(trace) # Square the trace

In [52]:
"""
Test for the purity function
"""
matrix_1 = np.array([[1/2,1/2],[1/2,1/2]])
matrix_2 = np.array([[3/4,1/4],[1/4,1/4]])

print("The purity of [[1/2,1/2],[1/2,1/2]] is {}".format(purity(matrix_1)))
print("The purity of [[3/4,1/4],[1/4,1/4]] is {}".format(purity(matrix_2)))


The purity of [[1/2,1/2],[1/2,1/2]] is 1.0
The purity of [[3/4,1/4],[1/4,1/4]] is 0.7499999999999999
